<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

## _*Relaxation and Decoherence*_ 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import qiskit
from qiskit.providers.aer.noise.errors.standard_errors import thermal_relaxation_error
from qiskit.providers.aer.noise import NoiseModel

from qiskit.ignis.characterization.coherence import T1Fitter, T2StarFitter, T2Fitter
from qiskit.ignis.characterization.coherence import t1_circuits, t2_circuits, t2star_circuits

# Generation of coherence circuits

In [ ]:
num_of_gates = (np.linspace(10, 300, 50)).astype(int)
gate_time = 0.1

# Note that it is possible to measure several qubits in parallel
qubits = [0, 2]

t1_circs, t1_xdata = t1_circuits(num_of_gates, gate_time, qubits)
t2star_circs, t2star_xdata, osc_freq = t2star_circuits(num_of_gates, gate_time, qubits, nosc=5)
t2echo_circs, t2echo_xdata = t2_circuits(num_of_gates, gate_time, qubits)
t2cpmg_circs, t2cpmg_xdata = t2_circuits(num_of_gates, gate_time, qubits, n_echos=5, alt_phase_echo=True)

# Backend execution

In [ ]:
backend = qiskit.Aer.get_backend('qasm_simulator')
shots = 200

# Let the simulator simulate the following times for qubits 0 and 2:
t_q0 = 25.0
t_q2 = 15.0

# Define T1 and T2 noise:
t1_noise_model = NoiseModel()
t1_noise_model.add_quantum_error(
    thermal_relaxation_error(t_q0, 2*t_q0, gate_time), 
    'id', [0])
t1_noise_model.add_quantum_error(
    thermal_relaxation_error(t_q2, 2*t_q2, gate_time), 
    'id', [2])

t2_noise_model = NoiseModel()
t2_noise_model.add_quantum_error(
    thermal_relaxation_error(np.inf, t_q0, gate_time, 0.5), 
    'id', [0])
t2_noise_model.add_quantum_error(
    thermal_relaxation_error(np.inf, t_q2, gate_time, 0.5), 
    'id', [2])


# Run the simulator
t1_backend_result = qiskit.execute(t1_circs, backend, shots=shots,
                                   noise_model=t1_noise_model).result()
t2star_backend_result = qiskit.execute(t2star_circs, backend, shots=shots,
                                       noise_model=t2_noise_model).result()
t2echo_backend_result = qiskit.execute(t2echo_circs, backend, shots=shots,
                                       noise_model=t1_noise_model).result()

# It is possible to split the circuits into multiple jobs and then give the results to the fitter as a list:
t2cpmg_backend_result1 = qiskit.execute(t2cpmg_circs[0:5], backend,
                                        shots=shots, noise_model=t2_noise_model).result()
t2cpmg_backend_result2 = qiskit.execute(t2cpmg_circs[5:], backend,
                                        shots=shots, noise_model=t2_noise_model).result()

# Analysis of results

In [ ]:
%matplotlib inline
# Fit the data to an exponential
# The correct answers are a=1, and c=0, and t1=25/15 for qubit 0/2
# The user does not know the correct answer exactly,
# so starts the fit from a different but close location

plt.figure(figsize=(15, 6))

initial_t1 = t1_0*1.2
initial_a = 1.0
initial_c = 0.0
#pass in a results list
fit = T1Fitter([backend_result1, backend_result2], xdata, qubits,
               fit_p0=[initial_a, initial_t1, initial_c],
               fit_bounds=([0, 0, -1], [2, initial_t1*2, 1]))

for i in range(2):
    ax = plt.subplot(1, 2, i+1)
    fit.plot(i, ax=ax)
    
print(fit.time())
plt.show()

Another feature is that we can rerun to get more statistics and easily add those results

In [ ]:
backend_result3 = qiskit.execute(circs, backend,
                                shots=shots, noise_model=noise_model).result()

fit.add_data(backend_result3)

In [ ]:
plt.figure(figsize=(15, 6))

for i in range(2):
    ax = plt.subplot(1, 2, i+1)
    fit.plot(i, ax=ax)
    
print(fit.time())
plt.show()

In [ ]:
%matplotlib inline
# Fit the data to an oscillator
# The correct answers are a=0.5, f=omega, phi=0, c=0.5, and t2=10/5 for qubit 0/2
# The user does not know the correct answer exactly,
# so starts the fit from a different but close location

plt.figure(figsize=(15, 6))

initial_t2 = t2_0*1.1
initial_a = 1
initial_c = 0
initial_f = osc_freq 
initial_phi = -np.pi/20

fit = T2StarFitter(backend_result, xdata, qubits,
                   fit_p0=[initial_a, initial_t2, initial_f, initial_phi, initial_c],
                   fit_bounds=([-0.5, 0, 0, -np.pi, -0.5],
                               [1.5, 2*t2_0, 2*osc_freq, np.pi, 1.5]))

for i in range(2):
    ax = plt.subplot(1, 2, i+1)
    fit.plot(i, ax=ax)
    
print(fit.time())
plt.show()

In [ ]:
%matplotlib inline
# Fit the data to an exponent
# The correct answers are a=1, c=0, and t2=10/5 for qubit 0/2
# The user does not know the correct answer exactly,
# so starts the fit from a different but close location

plt.figure(figsize=(15, 6))

initial_t2 = t2_0*1.1
initial_a = 1
initial_c = 0

fit = T2Fitter(backend_result, xdata, qubits,
               fit_p0=[initial_a, initial_t2, initial_c],
               fit_bounds=([-0.5, 0, -0.5],
                           [1.5, 2*t2_0, 1.5]))

for i in range(2):
    ax = plt.subplot(1, 2, i+1)
    fit.plot(i, ax=ax)
    
print(fit.time())
plt.show()

In [ ]:
%matplotlib inline
# Fit the data to an exponent
# The correct answers are a=1, c=0, and t2=10/5 for qubit 0/2
# The user does not know the correct answer exactly,
# so starts the fit from a different but close location

plt.figure(figsize=(15, 6))

initial_t2 = t2_0*1.1
initial_a = 1
initial_c = 0

fit = T2Fitter(backend_result, xdata, qubits,
               fit_p0=[initial_a, initial_t2, initial_c],
               fit_bounds=([-0.5, 0, -0.5],
                           [1.5, 2*t2_0, 1.5]))

for i in range(2):
    ax = plt.subplot(1, 2, i+1)
    fit.plot(i, ax=ax)
    
print(fit.time())
plt.show()